## pretrain unsupervised sentence transformer model
* on all text
* unsup methods only


`torch.cuda.empty_cache()`

In [1]:
import pandas as pd
import seaborn as sns
pd.set_option("display.precision", 4)
pd.set_option('display.width', 1024)
pd.set_option('display.max_colwidth', None)
%matplotlib inline 

from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score, cross_val_predict
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import classification_report, roc_auc_score  
# https://stackoverflow.com/questions/53784971/how-to-disable-convergencewarning-using-sklearn
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

from sentence_transformers import SentenceTransformer, util ## versions error? 

In [2]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, evaluation
import torch
torch.cuda.is_available()
from torch.utils.data import DataLoader
import torch.utils.data as data_utils
## https://stackoverflow.com/questions/50307707/convert-pandas-dataframe-to-pytorch-tensor
from transformers import AutoTokenizer, AutoModel
## example from : https://www.sbert.net/docs/training/overview.html
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses 

from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses, SentencesDataset ## https://www.sbert.net/docs/package_reference/losses.html + SentencesDataset
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentencesDataset, losses ## MultipleNegativesRankingLoss
## https://www.sbert.net/docs/package_reference/losses.html#multiplenegativesrankingloss
# from sentence_transformers.readers import InputExample
import logging
import os
import gzip
from torch.utils.data import DataLoader
from datetime import datetime
import sys

In [3]:
%%time
df = pd.read_csv("df_text.csv.gz").sample(frac=1).drop_duplicates()
df

Wall time: 2.39 s


,0
380151,Academy Award winner Meryl Streep. Betcha can't have just one!
433877,Next on ESPN2: The World Series of A man in yoga pants with a ponytail and feather earrings.
769229,"Unfortunately, no one can be told what A homoerotic volleyball montage is. You have to experience it for yourself."
5680,"Don't worry, kid. It gets better. I've been living with A sales team of clowns and pedophiles for 20 years."
812414,Hulu's new reality show features twelve hot singles living with Sucking the caviar straight out of a fish's pussy.
...,...
263092,"Oh, whoops! These are not the right pants for A bass drop so huge it tears the starry vault asunder to reveal the face of God."
832460,You Won't Believe These 15 Hilarious Becoming the President of the United States Bloopers!
838690,There ain't no problem a little hard liquor and One titty hanging out can't fix.
42418,"Armani suit: $1,000. Dinner for two at that swanky restaurant: $300. The look on her face when you surprise her with How good this cantaloupe feels on my penis: priceless."


In [4]:
df["0"].str.split().str.len().describe() # min 1, max 57

count    857056.0000
mean         15.8096
std           6.1033
min           1.0000
25%          11.0000
50%          15.0000
75%          19.0000
max          57.0000
Name: 0, dtype: float64

In [5]:
## drop mostly answers that have 1-2 words
df = df.loc[(df["0"].str.split().str.len()>4) & (df["0"].str.len()>14)].drop_duplicates()
print(df.shape[0])

850951


## Sentence transformers model(s)
* Sentence bert (NLI? semtantic similarity?) 
https://www.sbert.net/docs/training/overview.html
https://www.sbert.net/docs/quickstart.html  , https://github.com/UKPLab/sentence-transformers



List of pretrained sentencebert models:  
* https://www.sbert.net/docs/pretrained_models.html
* all-MiniLM-L12-v2
* all-MiniLM-L6-v2
* `stsb-distilroberta-base-v2` - 305M sized model - semantic similarity
* `nli-distilroberta-base-v2` - NLI 
* `average_word_embeddings_glove.6B.300d`, `average_word_embeddings_glove.840B.300d` - glove/w2v embeddings

In [6]:
torch.cuda.empty_cache() ## clear memory on rerun

In [7]:
model_name = "all-MiniLM-L12-v2"
# model_name ='output/cah_tsdae-model'

## "all-MiniLM-L12-v2"
## "all-MiniLM-L6-v2"
## paraphrase-MiniLM-L6-v2

In [8]:
## pretrained weights - download vis program often fails, easier to download externally from ftp
# model = SentenceTransformer('stsb-distilroberta-base-v2') ## was distilbert-base-nli-mean-tokens
# model = SentenceTransformer("nli-distilroberta-base-v2",device="cuda")
# model = SentenceTransformer("paraphrase-MiniLM-L6-v2") ## "paraphrase-MiniLM-L12-v2"

model = SentenceTransformer(model_name,device="cuda")

# model = SentenceTransformer("./stsb-distilroberta-base-v2",device="cuda")

# model = SentenceTransformer("./nli-mpnet-base-v2",device="cuda")

In [9]:
model.max_seq_length # 256

128

In [10]:
model.tokenize(["The fault, dear Brutus, is not in our stars, but in Meaningless sex."])

{'input_ids': tensor([[  101,  1996,  6346,  1010,  6203,  7987, 23998,  1010,  2003,  2025,
           1999,  2256,  3340,  1010,  2021,  1999, 25120,  3348,  1012,   102]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [11]:
len(model.tokenize(["The fault, dear Brutus, is not in our stars, but in Meaningless sex."])["input_ids"])

1

In [12]:
model.get_sentence_embedding_dimension() # 384

384

## TSDAE pretraining
https://www.sbert.net/examples/unsupervised_learning/TSDAE/README.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/unsupervised_learning/TSDAE/train_tsdae_from_file.py
* similar idea to MLM
* Doesn't use pairs data or the like


* L6, batch_size=64, amp : 33 minutes per tsdae epoch

In [13]:
# from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("sentence-transformers/paraphrase-MiniLM-L12-v2")

In [14]:
# # Define your sentence transformer model using CLS/mean (?) pooling
# model_name = "sentence-transformers/all-MiniLM-L12-v2"
# word_embedding_model = models.Transformer(model_name)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'mean')
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [ ]:
# # Define a list with sentences (1k - 100k sentences)
# train_sentences = ["Your set of sentences",
#                    "Model will automatically add the noise", 
#                    "And re-construct it",
#                    "You should provide at least 1k sentences"]

# Create the special denoising dataset that adds noise on-the-fly
train_dataset = datasets.DenoisingAutoEncoderDataset(df["0"].values)

# DataLoader to batch your data
train_dataloader = DataLoader(train_dataset, batch_size=40, shuffle=True)

# Use the denoising auto-encoder loss
train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

# Call the fit method
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=7, ## ~1 hour per epoch
    warmup_steps=2000,
    weight_decay=0,
    scheduler='constantlr',
    optimizer_params={'lr': 6e-5},
    show_progress_bar=True,
    use_amp=True,
    checkpoint_save_total_limit=5,
    output_path="output",
)

model.save('output/cah_tsdae-model')

In [16]:
# # Define your sentence transformer model using CLS pooling
# model_name = 'bert-base-uncased'
# word_embedding_model = models.Transformer(model_name)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 'cls')
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# # Define a list with sentences (1k - 100k sentences)
# train_sentences = ["Your set of sentences",
#                    "Model will automatically add the noise", 
#                    "And re-construct it",
#                    "You should provide at least 1k sentences"]

# # Create the special denoising dataset that adds noise on-the-fly
# train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

# # DataLoader to batch your data
# train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# # Use the denoising auto-encoder loss
# train_loss = losses.DenoisingAutoEncoderLoss(model, decoder_name_or_path=model_name, tie_encoder_decoder=True)

# # Call the fit method
# model.fit(
#     train_objectives=[(train_dataloader, train_loss)],
#     epochs=1,
#     weight_decay=0,
#     scheduler='constantlr',
#     optimizer_params={'lr': 3e-5},
#     show_progress_bar=True
# )

# model.save('output/tsdae-model')